In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=43e820e76adcef7c747c3e0a521227d640a87105d44548c211508fe08f234876
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [4]:
text_column_name = "tokens"
label_column_name = "ner_tags"
label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)

{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}


In [5]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/IndicNER', num_labels=num_labels )

2024-03-13 10:07:05.223491: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 10:07:05.223585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 10:07:05.349761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
model=model.to("cuda")

In [7]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
# # Get the length of the original training set
# original_train_size = len(raw_datasets["train"])

# # Calculate the number of samples to keep for 20% of the original training set
# new_train_size = int(original_train_size * 0.05)
train_dataset = raw_datasets["train"].select(range(20000))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

In [9]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [10]:
data_collator = DataCollatorForTokenClassification(tokenizer)


In [11]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [ ]:
# args=TrainingArguments(
#     output_dir='output_dir',
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     evaluation_strategy = "epoch",
#     learning_rate=5e-5,
#     weight_decay=0.02
# )

# args=TrainingArguments(
#     output_dir='output_dir',
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     evaluation_strategy = "epoch",
#     learning_rate=3e-5)

# args=TrainingArguments(
#     output_dir='output_dir',
#     per_device_train_batch_size=12,
#     per_device_eval_batch_size=12,
#     num_train_epochs=3,
#     evaluation_strategy = "epoch",
#     learning_rate=2e-5)

In [12]:
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy = "epoch",
    learning_rate=4e-5,
    weight_decay=0.01
)

In [13]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [14]:
trainer.args

TrainingArguments(
_n_gpu=2,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

In [15]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.151300,0.170940,0.810182,0.855478,0.832214,10213,0.691108,0.679031,0.685016,9786,0.806103,0.839894,0.822652,10568,0.772302,0.793601,0.782807,0.947466
2,0.104300,0.180633,0.820590,0.855380,0.837624,10213,0.676650,0.692622,0.684543,9786,0.805606,0.832135,0.818656,10568,0.769752,0.795237,0.782287,0.947105
3,0.077700,0.202834,0.813749,0.853030,0.832927,10213,0.675241,0.686695,0.680920,9786,0.802662,0.827498,0.814891,10568,0.766235,0.790951,0.778397,0.946471


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [16]:
Final_metrics = trainer.evaluate()

trainer.log_metrics("eval", Final_metrics)

***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.8329
  eval_LOC_number         =      10213
  eval_LOC_precision      =     0.8137
  eval_LOC_recall         =      0.853
  eval_ORG_f1             =     0.6809
  eval_ORG_number         =       9786
  eval_ORG_precision      =     0.6752
  eval_ORG_recall         =     0.6867
  eval_PER_f1             =     0.8149
  eval_PER_number         =      10568
  eval_PER_precision      =     0.8027
  eval_PER_recall         =     0.8275
  eval_loss               =     0.2028
  eval_overall_accuracy   =     0.9465
  eval_overall_f1         =     0.7784
  eval_overall_precision  =     0.7662
  eval_overall_recall     =      0.791
  eval_runtime            = 0:05:02.85
  eval_samples_per_second =     44.443
  eval_steps_per_second   =       2.78


Here we can see that macro f1 score /Overall f1 score on validation data set is 0.7784

In [17]:
model.save_pretrained('Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL')

In [18]:
tokenizer.save_pretrained("tokenizer_indicNER_BatchSize=8_FINAL")

('tokenizer_indicNER_BatchSize=8_FINAL/tokenizer_config.json',
 'tokenizer_indicNER_BatchSize=8_FINAL/special_tokens_map.json',
 'tokenizer_indicNER_BatchSize=8_FINAL/vocab.txt',
 'tokenizer_indicNER_BatchSize=8_FINAL/added_tokens.json',
 'tokenizer_indicNER_BatchSize=8_FINAL/tokenizer.json')

In [19]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [20]:
import json
config = json.load(open("/kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL/config.json"))

In [21]:
config["id2label"] = id2label
config["label2id"] = label2id

In [22]:
json.dump(config, open("/kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL/config.json","w"))

In [23]:
model = AutoModelForTokenClassification.from_pretrained('/kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL')

In [24]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [35]:
!zip -r IndicNERTuned_20000_BatchSize8weightdecay_FINAL.zip /kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL

  adding: kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL/ (stored 0%)
  adding: kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL/config.json (deflated 50%)
  adding: kaggle/working/Finetuned_indicNER_with_20000_sentences_BatchSize=8_FINAL/model.safetensors (deflated 7%)


In [36]:
!zip -r IndicNER_tokenizer_20000_BatchSize8_FINAL.zip /kaggle/working/tokenizer_indicNER_BatchSize=8_FINAL

  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_FINAL/ (stored 0%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_FINAL/vocab.txt (deflated 48%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_FINAL/tokenizer.json (deflated 69%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_FINAL/special_tokens_map.json (deflated 80%)
  adding: kaggle/working/tokenizer_indicNER_BatchSize=8_FINAL/tokenizer_config.json (deflated 75%)


In [25]:
test_dataset = raw_datasets["test"]
tokenize_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
predictions, labels, metrics = trainer.predict(tokenize_test_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]

In [27]:
print(metrics)

{'test_loss': 0.16923412680625916, 'test_LOC_precision': 0.8093699515347335, 'test_LOC_recall': 0.8159609120521173, 'test_LOC_f1': 0.8126520681265207, 'test_LOC_number': 614, 'test_ORG_precision': 0.6560509554140127, 'test_ORG_recall': 0.7847619047619048, 'test_ORG_f1': 0.7146574154379879, 'test_ORG_number': 525, 'test_PER_precision': 0.8329355608591885, 'test_PER_recall': 0.8835443037974684, 'test_PER_f1': 0.8574938574938575, 'test_PER_number': 790, 'test_overall_precision': 0.7726618705035971, 'test_overall_recall': 0.8351477449455676, 'test_overall_f1': 0.8026905829596414, 'test_overall_accuracy': 0.95385311416076, 'test_runtime': 19.76, 'test_samples_per_second': 43.877, 'test_steps_per_second': 2.783}


In [28]:
print('macro f1 score is ::',metrics['test_overall_f1'])

macro f1 score is :: 0.8026905829596414


so macro f1 score of test data set is 0.8026905829596414

In [29]:
print(lang_specific_results)

{'Precision': 0.7726618705035971, 'Recall': 0.8351477449455676, 'F1': 0.8026905829596414}


In [30]:
predictions, labels, metrics = trainer.predict(train_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]

In [31]:
print(metrics)
print('\n')
print(lang_specific_results)

{'test_loss': 0.05327076464891434, 'test_LOC_precision': 0.90600624024961, 'test_LOC_recall': 0.9391550434606832, 'test_LOC_f1': 0.9222828784119107, 'test_LOC_number': 14841, 'test_ORG_precision': 0.862873399715505, 'test_ORG_recall': 0.8615253515125693, 'test_ORG_f1': 0.8621988486958994, 'test_ORG_number': 14082, 'test_PER_precision': 0.9137515842839037, 'test_PER_recall': 0.9234661201485846, 'test_PER_f1': 0.9185831687583615, 'test_PER_number': 15614, 'test_overall_precision': 0.895298956306386, 'test_overall_recall': 0.9091092799245571, 'test_overall_f1': 0.9021512683682222, 'test_overall_accuracy': 0.982365987754592, 'test_runtime': 449.9107, 'test_samples_per_second': 44.453, 'test_steps_per_second': 2.778}


{'Precision': 0.895298956306386, 'Recall': 0.9091092799245571, 'F1': 0.9021512683682222}


so macro f1 score of test data set is 0.9021512683682222